In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from pprint import pprint

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("http://www.infopub.co.kr/new/include/detail.asp?sku=05000274",),
)
docs = loader.load()

print(len(docs))
print(docs[0])
pprint(docs[0].metadata)

USER_AGENT environment variable not set, consider setting it to identify your requests.


1
page_content='\n\n\n\n\n[정보문화사] IT 도서의 새로운 패러다임\n\n\n\n\nCJ스타일베너[좌슬라이드]\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n파이썬 머신러닝 판다스 데이터 분석 개정판\r\n\n\n\n저자:\n\n오승환\n\n\n\n역자:\n\n\n\n\n구분:\n\r\n\t\r\n\t\r\n       국내서\r\n    \r\n\t\r\n\t\n\n\n발행일:\n2024년 6월 25일\n\n\n정가:\n30,000원\n\n\n페이지:\n584 페이지\n\n\nISBN:\n978-89-5674-980-8\n\n\n출판사:\n\n\r\n\t\t\t\t정보문화사\r\n\r\n\t\n\n\n판형:\n187×235\n\n\n\n난이도:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n책소개\n저자소개\n미리보기\n오탈자\n보도자료실\n샘플자료 다운로드\n\n\n\n데이터 과학자가 되기 위한 첫걸음!어려운 이론은 최소화하고, 예제 코드를 따라 하며 자연스럽게 사용법에 익숙해지도록 안내하는 데이터 분석 입문서다. 데이터 분석을 처음 배우는 입문자의 입장에서 고급 이론과 데이터 분석 도구를 함께 배우는 것의 어려움을 아는 저자가, 데이터 분석에 필요한 필수 라이브러리를 소개하고 설치부터 예제 코드까지 따라 할 수 있게 구성했다. 개념 이해를 돕기 위해 다이어그램 등 풍부한 도식화도 적극 활용했다. 1판에서 큰 인기를 얻어 준비된 이번 개정판에서는 저자가 실무에서 쌓은 경험을 자연스럽게 녹이기

In [2]:
# HugoingFace Embeddings를 다운로드
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
)

# HugoingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Token 수를 기준으ㄹ 문서를 청크 단위로 분할
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 120,
    chunk_overlap  = 10,
)

split_docs = text_splitter.split_documents(docs)
print(len(split_docs))
print(split_docs[0])

36
page_content='[정보문화사] IT 도서의 새로운 패러다임\n\n\n\n\nCJ스타일베너[좌슬라이드]' metadata={'source': 'http://www.infopub.co.kr/new/include/detail.asp?sku=05000274', 'title': '[정보문화사] IT 도서의 새로운 패러다임', 'language': 'No language found.'}


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from autorag.data.corpus import langchain_documents_to_parquet
import os 

base_dir = 'corpus_data'
os.makedirs(base_dir, exist_ok=True)


corpus_df = langchain_documents_to_parquet(split_docs, 'corpus_data/pandas_book.parquet')
corpus_df.head()

c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


,doc_id,contents,metadata
0,5e7d4826-3691-4362-81e3-2cc4f85a3f71,[정보문화사] IT 도서의 새로운 패러다임\n\n\n\n\nCJ스타일베너[좌슬라이드],{'source': 'http://www.infopub.co.kr/new/inclu...
1,65019448-d953-4f4c-bea6-0b02598cc037,파이썬 머신러닝 판다스 데이터 분석 개정판\r\n\n\n\n저자:,{'source': 'http://www.infopub.co.kr/new/inclu...
2,db0d30c3-5fbb-4702-9f66-7920d82780bf,저자:\n\n오승환\n\n\n\n역자:\n\n\n\n\n구분:\n\r\n\t\r\n...,{'source': 'http://www.infopub.co.kr/new/inclu...
3,92ce81eb-4c63-4b0f-b0bd-e94ff84edf62,난이도:,{'source': 'http://www.infopub.co.kr/new/inclu...
4,17c12c32-187b-4d7a-bd10-67b82cf5e0b7,책소개\n저자소개\n미리보기\n오탈자\n보도자료실\n샘플자료 다운로드,{'source': 'http://www.infopub.co.kr/new/inclu...


In [6]:
import pandas as pd

from autorag.data.qacreation import generate_qa_llama_index, make_single_content_qa
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI


prompt = """
Generate question and answer pairs for the given passage.

Passage:
{{text}}

Number of questions to generate: {{num_questions}}

Example:
[Q]: 이 책의 글쓴이는 누구인가요?
[A]: 저자의 이름은 홍길동입니다.

Result:
"""

corpus_df = pd.read_parquet('corpus_data/pandas_book.parquet')

llm = OpenAI(model='gpt-4o', temperature=0.1)

qa_df = make_single_content_qa(corpus_df, content_size=36, qa_creation_func=generate_qa_llama_index,
                            llm=llm, prompt=prompt, question_num_per_content=1)

qa_df.head()

[07/12/24 10:49:31] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=69714;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=791576;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=514037;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=226929;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=357314;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=612251;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[07/12/24 10:49:32] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=236523;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=520545;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=823259;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=82347;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

[07/12/24 10:49:42] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=342622;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=450417;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=832571;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=643776;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=198903;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=798578;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=761734;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=519152;file://c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

,qid,retrieval_gt,query,generation_gt
0,8ec9448a-31fc-458d-8e8e-06e0d633ee23,[[5448c43e-315e-471b-90db-d10a602d1370]],본문 소스 코드의 등록일은 언제인가요?,[등록일은 2024년 7월 3일입니다.]
1,e79efaa5-76b6-491b-aa60-13e41eb25e19,[[16d2b087-e72d-4458-9429-65d8ccef2a1c]],이 책에서 추가된 주요 내용은 무엇인가요?,[시계열 데이터의 효율적인 처리와 분석을 위한 다양한 기법이 추가되었습니다.]
2,ea10a90c-c4c5-4280-9251-e73e646e505b,[[372d59aa-4512-4b0b-a347-befd53a0653e]],"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 작성일은 언제인가요?",[작성일은 2024년 4월 24일입니다.]
3,1410cc17-27d0-4cfe-9b56-06963bcd74e9,[[dc4e70b6-ce74-456c-89af-644222a4feb3]],"""가장 쉬운 이모티콘 만들기 2판"" 보도자료는 언제 발표되었나요?","[""가장 쉬운 이모티콘 만들기 2판"" 보도자료는 2023년 8월 30일에 발표되었습..."
4,332ebf24-9d3e-42e5-9de5-d42176c00626,[[d70c285b-3a6a-4e28-98de-8d9e23dc73d1]],데이터 변화를 효과적으로 분석하기 위해 어떤 조치를 취했나요?,[시계열 데이터 처리 강화와 데이터 전처리 기법을 다양화했습니다.]


In [10]:
qa_df.to_excel('corpus_data/pandas_book_qa.xlsx', index=False)

In [11]:
# 직접 수정한 QA 데이터를 불러옴
qa_data = pd.read_excel('corpus_data/pandas_book_qa.xlsx')
qa_data.head()

,qid,retrieval_gt,query,generation_gt
0,b5584976-bab3-4802-9cbc-dae30e66a3be,[['dd7329e5-3132-434c-adb1-db6ac94d6ae8']],본문 소스 코드의 등록일은 언제인가요?,['등록일은 2024년 7월 3일입니다.']
1,4aa93437-241e-4614-aa58-d1d17cc3ebde,[['891af9e1-fa65-4f6b-81ac-f5a162c1ccc8']],이 책에서 추가된 주요 내용은 무엇인가요?,['시계열 데이터의 효율적인 처리와 분석을 위한 다양한 기법이 추가되었습니다.']
2,1bdaff50-a4b2-43d2-af42-b937bfd6261e,[['bc5e7685-32cd-44a7-9d88-7acde60c1252']],"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표 날짜는 언제인가요?",['발표 날짜는 2024년 4월 24일입니다.']
3,4fa63920-0aca-48dd-8250-1331e82c48d6,[['b0dea12b-f1f1-4ba5-95a2-6f62f42f764e']],"""가장 쉬운 이모티콘 만들기 2판"" 보도자료는 언제 발표되었나요?","['""가장 쉬운 이모티콘 만들기 2판"" 보도자료는 2023년 8월 30일에 발표되었..."
4,54edcfff-6e72-4a4b-90e8-e807aac3aa89,[['4638ea57-adbc-49ac-ad16-46c38cecd190']],시계열 데이터 처리를 어떻게 강화했나요?,"['시계열 데이터 처리 강화와 더불어, 데이터 전처리 기법을 더욱 다양화했습니다.']"


In [12]:
import ast
for col in ['retrieval_gt', 'generation_gt']:
    qa_data[col] = qa_data[col].apply(ast.literal_eval)

qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qid            36 non-null     object
 1   retrieval_gt   36 non-null     object
 2   query          36 non-null     object
 3   generation_gt  36 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


In [13]:
qa_data = qa_data[['query', 'generation_gt']]
qa_data.columns = ['question', 'ground_truth']
qa_data.head()

,question,ground_truth
0,본문 소스 코드의 등록일은 언제인가요?,[등록일은 2024년 7월 3일입니다.]
1,이 책에서 추가된 주요 내용은 무엇인가요?,[시계열 데이터의 효율적인 처리와 분석을 위한 다양한 기법이 추가되었습니다.]
2,"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표 날짜는 언제인가요?",[발표 날짜는 2024년 4월 24일입니다.]
3,"""가장 쉬운 이모티콘 만들기 2판"" 보도자료는 언제 발표되었나요?","[""가장 쉬운 이모티콘 만들기 2판"" 보도자료는 2023년 8월 30일에 발표되었습..."
4,시계열 데이터 처리를 어떻게 강화했나요?,"[시계열 데이터 처리 강화와 더불어, 데이터 전처리 기법을 더욱 다양화했습니다.]"


In [14]:
qa_data['ground_truth'] = qa_data['ground_truth'].apply(lambda x: x[0] )
qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke(x) )
qa_data['contexts'] = qa_data['question'].apply(lambda x: [d.page_content for d in retriever.get_relevant_documents(x)] )
qa_data.head()

C:\Users\kim_h\AppData\Local\Temp\ipykernel_25588\1755535353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_data['ground_truth'] = qa_data['ground_truth'].apply(lambda x: x[0] )
C:\Users\kim_h\AppData\Local\Temp\ipykernel_25588\1755535353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke(x) )
c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method

,question,ground_truth,answer,contexts
0,본문 소스 코드의 등록일은 언제인가요?,등록일은 2024년 7월 3일입니다.,본문 소스 코드의 등록일은 2024년 7월 3일입니다.,[비전공자와 입문자의 마음을 사로잡은 강의가 개정판으로 돌아왔다. 최신 버전인 Pa...
1,이 책에서 추가된 주요 내용은 무엇인가요?,시계열 데이터의 효율적인 처리와 분석을 위한 다양한 기법이 추가되었습니다.,이 책에서 추가된 주요 내용은 Pandas 2.0 버전에서 추가되거나 변경된 내용입...,[비전공자와 입문자의 마음을 사로잡은 강의가 개정판으로 돌아왔다. 최신 버전인 Pa...
2,"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표 날짜는 언제인가요?",발표 날짜는 2024년 4월 24일입니다.,"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표 날짜는 2024년 4월 2...",[330\n\n[보도자료] 예제가 가득한 챗GPT 프롬프트 길라잡이\n운영자\n20...
3,"""가장 쉬운 이모티콘 만들기 2판"" 보도자료는 언제 발표되었나요?","""가장 쉬운 이모티콘 만들기 2판"" 보도자료는 2023년 8월 30일에 발표되었습니다.","""가장 쉬운 이모티콘 만들기 2판"" 보도자료는 2023년 8월 30일에 발표되었습니다.",[5674-930.zip(4293KB)\n\n\n\n \n\n\n\r\n ...
4,시계열 데이터 처리를 어떻게 강화했나요?,"시계열 데이터 처리 강화와 더불어, 데이터 전처리 기법을 더욱 다양화했습니다.","시계열 데이터 처리를 강화하는 방법 중 하나로, 데이터 전처리 기법을 다양화하였습니...",[대폭 추가해 시간에 따른 데이터 변화를 효과적으로 분석할 수 있도록 했다. 또한 ...


In [15]:
from datasets import Dataset 

dataset = Dataset.from_pandas(qa_data)

dataset[0]

{'question': '본문 소스 코드의 등록일은 언제인가요?',
 'ground_truth': '등록일은 2024년 7월 3일입니다.',
 'answer': '본문 소스 코드의 등록일은 2024년 7월 3일입니다.',
 'contexts': ['비전공자와 입문자의 마음을 사로잡은 강의가 개정판으로 돌아왔다. 최신 버전인 Pandas 2.0 버전에서 추가되거나 변경된 내용을 포함하여 시계열 데이터 처리 등의 기법과',
  '데이터 과학자가 되기 위한 첫걸음!어려운 이론은 최소화하고, 예제 코드를 따라 하며 자연스럽게 사용법에 익숙해지도록 안내하는 데이터 분석 입문서다. 데이터 분석을 처음',
  '지은이 | 오승환과학고, 서울대를 졸업하고, 중 국 C KGSB MBA, FRM(미국 재무위험관리사) 자격을 보유하고 있다. 국내 주요 금융기관과 대기업에서 기업분석 및 전략기획 업무를 담당했다. IT 비전공자이지만 데이터 분석과 인공지능을 독학으로 익혔고, 현재 핀테크 스타트업 대표이자 인공지능 강사로 활발하게 활동하고 있다. 유튜브 <판다스 스튜디오>를 운영하며, 저서로는 『파이썬 딥러닝 텐서플로』 등이 있다.',
  '5674-919.zip(9267KB)\n\n\n\n \n\n\n\n\n\n\n\n\n자료명\n등록일\n다운로드\n\n\n본문 소스 코드\n2024-07-03\n05000274.zip']}

In [16]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [17]:
from ragas import evaluate
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

langchain_llm =  ChatOllama(model="gemma2")
langchain_embeddings = OllamaEmbeddings(model="gemma2")

result = evaluate(
    dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm, 
    embeddings=langchain_embeddings,
    raise_exceptions=False,
)

result

Evaluating:   0%|          | 0/144 [00:00<?, ?it/s]